In [45]:
import json
import logging
import pandas as pd
import numpy as np
import os
from pathlib import Path
from yfpy import YahooFantasySportsQuery
from yfpy import get_logger
from yfpy.utils import (
    unpack_data,
    dict_to_list,
    complex_json_handler,
    flatten_json_dict_list,
    flatten_to_list,
    reformat_json_list,
)
from dotenv import load_dotenv
from gspread_pandas import Spread

from utils import (
    get_season,
    league_season_info,
    google_sheet_upload,
    sql_upload_table,
    sql_grab_table,
)


load_dotenv()
pd.set_option("display.max_colwidth", 999)

LOGGER = get_logger(__name__)
LOG_OUTPUT = False
logging.getLogger("yfpy.query").setLevel(level=logging.INFO)

PATH = Path.cwd().parents[0]
DATA_DIR = PATH / "data"

SEASON = get_season()
NFL_DATES_DF, LEAGUE_ID_DF = league_season_info()

# TODAY = np.datetime64("today", "D")
TODAY = np.datetime64("2021-09-28")
NFL_WEEK = NFL_DATES_DF["Week"][
    (NFL_DATES_DF["End_Date"] >= TODAY) & (NFL_DATES_DF["Start_Date"] <= TODAY)
].values[0]
LEAGUE_ID = LEAGUE_ID_DF[LEAGUE_ID_DF["season"] == SEASON]["league_ID"].values[0]
GAME_ID = LEAGUE_ID_DF[LEAGUE_ID_DF["season"] == SEASON]["game_ID"].values[0]
WEEKS = list(range(NFL_WEEK, 0, -1))

CONSUMER_KEY = os.getenv("yahoo_client_id")
CONSUMER_SECRET = os.getenv("yahoo_client_secret")

try:
    yahoo_query = YahooFantasySportsQuery(
        auth_dir=PATH,
        league_id=LEAGUE_ID,
        game_id=GAME_ID,
        game_code="nfl",
        offline=False,
        all_output_as_json=False,
        consumer_key=CONSUMER_KEY,
        consumer_secret=CONSUMER_SECRET,
        browser_callback=True,
    )

except Exception as e:
    print(e)

In [ ]:
class league_season_data(object):
    def __init__(
        self,
        auth_dir: Path,
        league_id: str,
        game_id: int = None,
        game_code: str = "nfl",
        offline: bool = False,
        all_output_as_json: bool = False,
        consumer_key: str = None,
        consumer_secret: str = None,
        browser_callback: bool = True,
    ):

        self._auth_dir = auth_dir
        self._consumer_key = consumer_key
        self._consumer_secret = consumer_secret
        self._browser_callback = browser_callback

        self.league_id = league_id
        self.game_id = game_id
        self.game_code = game_code

        self.offline = offline
        self.all_output_as_json = all_output_as_json

    def _yahoo_query(self):

        try:
            yahoo_query = YahooFantasySportsQuery(
                auth_dir=self._auth_dir,
                league_id=self.league_id,
                game_id=self.game_id,
                game_code=self.game_code,
                offline=self.offline,
                all_output_as_json=self.all_output_as_json,
                consumer_key=self._consumer_key,
                consumer_secret=self._consumer_secret,
                browser_callback=self._browser_callback,
            )

            return yahoo_query

        except Exception as e:
            print(e)

    def get_league_metadata(self):
        pass

    def get_league_settings(self):
        pass

    def get_league_teams(self):
        pass

    def get_league_players(self):
        pass

    def get_league_draft_results(self):
        pass

    def get_league_scoreboard_by_week(self):
        pass

    def get_team_metadata(self):
        pass

    def get_team_stats_by_week(self):
        pass

    def get_team_roster_by_week(self):
        pass

    def get_team_draft_results(self):
        pass

    def get_team_matchups(self):
        pass

    def get_player_stats_by_week(self):
        pass

    def get_player_percent_owned_by_week(self):
        pass

    def get_player_draft_analysis(self):
        pass

    def all_game_keys(self):
        data = unpack_data(yahoo_query.get_all_yahoo_fantasy_game_keys())
        game_keys = pd.DataFrame()
        for row in data:
            row = pd.DataFrame(complex_json_handler(row["game"]), index=[0])
            game_keys = pd.concat([game_keys, row])
        game_keys.reset_index(drop=True, inplace=True)
        game_keys = game_keys[game_keys["season"] >= 2012]
        sql_upload_table(
            dataframe=game_keys,
            table_name="GameKeys",
            data_schema="dbo",
            chunksize=500,
            if_exists="replace",
            index=False,
        )

    def all_nfl_weeks(self):
        game_keys = sql_grab_table("GameKeys")
        game_id = list(game_keys["game_id"])
        weeks = pd.DataFrame()
        for g in game_id:
            data = yahoo_query.get_game_weeks_by_game_id(g)
            for i in data:
                row = pd.json_normalize(complex_json_handler(i["game_week"]))
                row["game_id"] = g
                weeks = pd.concat([weeks, row])
        weeks.rename(columns={"display_name": "week"}, inplace=True)
        sql_upload_table(
            dataframe=weeks,
            table_name="NFLWeeks",
            data_schema="dbo",
            chunksize=500,
            if_exists="replace",
            index=False,
        )